In [1]:
import os

# 获取当前工作目录
current_dir = os.getcwd()
print("当前工作目录：", current_dir)
 
# 切换到上一层目录
parent_dir = os.path.dirname(current_dir)
os.chdir(parent_dir)
print("切换后的目录：", parent_dir)

当前工作目录： c:\Users\zhaoxs3\Documents\GitHub\NEMESIS\unit_test
切换后的目录： c:\Users\zhaoxs3\Documents\GitHub\NEMESIS


In [2]:
import QuantLib as ql
import numpy as np
import pandas as pd

from devlib.market.curves.overnight_index_curves import Sofr, Estr
from devlib.market.curves.credit_curve_generator import MarketDataCreditCurve, CdsConfig
from devlib.products.credit.cds import Cds

from devlib.utils.ql_date_utils import ql_date_str
from devlib.utils.cds_utils import cds_maturity_date, calendar_5u

In [3]:
from nemesis.products.credit import *
from nemesis.utils import *

####################################################################
#  NEMESIS ALPHA Version 0.1.0 - This build: 24 Jan 2025 at 10:42 #
####################################################################



### Test Case 1

In [6]:
today = ql.Date(1,11,2024)
ql.Settings.instance().evaluationDate = today

mkt_file_path = './unit_test/data/ICVS530_curve_data_20241101.xlsx'
swap_mkt_data = pd.read_excel(mkt_file_path, sheet_name='swap')
discount_curve = Estr(today, swap_mkt_data=swap_mkt_data)

entity = 'EUROPE'
mkt_file_path = './unit_test/data/credit_curve_data_20241101.xlsx'
cds_type = 'index'
cds_data = pd.read_excel(mkt_file_path, sheet_name=cds_type)

cds_config = CdsConfig(recovery_rate = 0.4,
                        daycount = ql.Actual360(),
                        calendar = ql.WeekendsOnly())
daycount = ql.Actual360()

credit_curve = MarketDataCreditCurve(today, entity, cds_data, discount_curve, cds_type, cds_config, daycount)

In [7]:
value_dt = Date(1,11,2024)
issuer_curve = QLCreditCurve(value_dt, credit_curve)

In [8]:
cds_contract = GeneralCDS(
    step_in_dt=value_dt,
    maturity_dt_or_tenor="10Y",
    running_cpn=0.01,
    notional=1e7,
    long_protect=True,
    freq_type=FrequencyTypes.QUARTERLY,
    dc_type=DayCountTypes.ACT_360,
    cal_type=CalendarTypes.WEEKEND,
    bd_type=BusDayAdjustTypes.FOLLOWING,
    dg_type=DateGenRuleTypes.BACKWARD
)

In [14]:
cds_contract.value(value_dt, issuer_curve)

{'dirty_pv': np.float64(-29153.04997832945),
 'clean_pv': np.float64(-17208.60553388507)}

In [10]:
cds_contract.credit_dv01(value_dt, issuer_curve, 0.4, bump=1)

{'dirty_pv': np.float64(-20522.885508437408), 'clean_pv': np.float64(-8578.441063993145)}
{'dirty_pv': np.float64(-37797.15060340916), 'clean_pv': np.float64(-25852.706158964662)}


np.float64(8637.132547485875)

In [16]:
cds_contract.interest_dv01(value_dt, issuer_curve, 0.4)

np.float64(8.566898120916449)

### Test Case 2

In [11]:
today = ql.Date(1,11,2024)
ql.Settings.instance().evaluationDate = today

mkt_file_path = './unit_test/data/ICVS531_curve_data_20241101.xlsx'
swap_mkt_data = pd.read_excel(mkt_file_path, sheet_name='swap')
discount_curve = Sofr(today, swap_mkt_data=swap_mkt_data)

entity = 'CHINAGOV'
mkt_file_path = './unit_test/data/credit_curve_data_20241101.xlsx'
cds_type = 'single_name'
cds_data = pd.read_excel(mkt_file_path, sheet_name=cds_type)

cds_config = CdsConfig(recovery_rate = 0.4,
                        daycount = ql.Actual360(),
                        calendar = calendar_5u())
daycount = ql.Actual360()

credit_curve = MarketDataCreditCurve(today, entity, cds_data, discount_curve, cds_type, cds_config, daycount)

In [12]:
value_dt = Date(1,11,2024)
issuer_curve = QLCreditCurve(value_dt, credit_curve)

In [13]:
cds_contract = GeneralCDS(
    step_in_dt=value_dt,
    maturity_dt_or_tenor="10Y",
    running_cpn=0.05,
    notional=1e7,
    long_protect=True,
    freq_type=FrequencyTypes.QUARTERLY,
    dc_type=DayCountTypes.ACT_360,
    cal_type=CalendarTypes.CDS,
    bd_type=BusDayAdjustTypes.FOLLOWING,
    dg_type=DateGenRuleTypes.BACKWARD
)

In [ ]:
cds_contract.risky_pv01(value_dt, issuer_curve)

In [9]:
cds_contract.value(value_dt, issuer_curve)

{'dirty_pv': np.float64(-3281982.6643738346),
 'clean_pv': np.float64(-3222260.4421516117)}

In [ ]:
accrual = cds_contract.running_cpn * cds_contract.notional * (rpv01["dirty_rpv01"] - rpv01["clean_rpv01"])
accrual

In [12]:
prot_pv = cds_contract.prot_leg_pv(value_dt, issuer_curve)
prot_pv

np.float64(773291.5969352175)

In [14]:
cds_contract.credit_dv01(value_dt, issuer_curve, 0.4, bump=1)

{'dirty_pv': np.float64(-3271472.5250306847), 'clean_pv': np.float64(-3211750.3028084626)}
{'dirty_pv': np.float64(-3292508.0872085625), 'clean_pv': np.float64(-3232785.8649863405)}


np.float64(10517.781088938937)

In [14]:
cds_contract.interest_dv01(value_dt, issuer_curve, 0.4)

np.float64(1502.1983877439052)

### Test Case 3

In [4]:
today = ql.Date(1,11,2024)
ql.Settings.instance().evaluationDate = today

mkt_file_path = './unit_test/data/ICVS531_curve_data_20241101.xlsx'
swap_mkt_data = pd.read_excel(mkt_file_path, sheet_name='swap')
discount_curve = Sofr(today, swap_mkt_data=swap_mkt_data)

entity = 'CHINAGOV'
mkt_file_path = './unit_test/data/credit_curve_data_20241101.xlsx'
cds_type = 'single_name'
cds_data = pd.read_excel(mkt_file_path, sheet_name=cds_type)

cds_config = CdsConfig(recovery_rate = 0.4,
                        daycount = ql.Actual360(),
                        calendar = calendar_5u())
daycount = ql.Actual360()

credit_curve = MarketDataCreditCurve(today, entity, cds_data, discount_curve, cds_type, cds_config, daycount)

In [5]:
value_dt = Date(1,11,2024)
issuer_curve = QLCreditCurve(value_dt, credit_curve)

In [6]:
cds_contract = GeneralCDS(
    step_in_dt=value_dt,
    maturity_dt_or_tenor=Date(1,11,2029),
    running_cpn=0.0,
    upfront_payment_dt=Date(6,11,2024),
    upfront_amount=3e5,    
    notional=1e7,
    long_protect=True,
    freq_type=FrequencyTypes.QUARTERLY,
    dc_type=DayCountTypes.ACT_360,
    cal_type=CalendarTypes.WEEKEND,
    bd_type=BusDayAdjustTypes.FOLLOWING,
    dg_type=DateGenRuleTypes.BACKWARD
)

In [7]:
cds_contract.value(value_dt, issuer_curve)

np.float64(-6439.825080876122)

In [8]:
cds_contract.credit_dv01(value_dt, issuer_curve, 0.4, bump=1)

np.float64(4377.223720230453)

In [9]:
cds_contract.interest_dv01(value_dt, issuer_curve, 0.4)

np.float64(-71.9812481815461)